# Soil Moisture: Access and Analysis
#### John Bolten <john.d.bolten@nasa.gov>
#### Original code author: Bin Fang <bf3fh@virginia.edu>

In [1]:
# Import required packages
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import glob
import h5py
import calendar
from osgeo import ogr
import fiona
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
from rasterio.transform import from_origin, Affine
from rasterio.windows import Window
from rasterio.crs import CRS
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import datetime
# Ignore runtime warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

#### Function 1: Subset the coordinates table of study area

In [2]:
def coordtable_subset(lat_input, lon_input, lat_extent_max, lat_extent_min, lon_extent_max, lon_extent_min):
    lat_output = lat_input[np.where((lat_input <= lat_extent_max) & (lat_input >= lat_extent_min))]
    row_output_ind = np.squeeze(np.array(np.where((lat_input <= lat_extent_max) & (lat_input >= lat_extent_min))))
    lon_output = lon_input[np.where((lon_input <= lon_extent_max) & (lon_input >= lon_extent_min))]
    col_output_ind = np.squeeze(np.array(np.where((lon_input <= lon_extent_max) & (lon_input >= lon_extent_min))))

    return lat_output, row_output_ind, lon_output, col_output_ind

#### Function 2: Subset and reproject the Geotiff data to WGS84 projection

In [3]:
def sub_n_reproj(input_mat, kwargs_sub, sub_window, output_crs):
    # Get the georeference and bounding parameters of subset image
    kwargs_sub.update({
        'height': sub_window.height,
        'width': sub_window.width,
        'transform': rasterio.windows.transform(sub_window, kwargs_sub['transform'])})

    # Generate subset rasterio dataset
    input_ds_subset = MemoryFile().open(**kwargs_sub)
    input_mat = np.expand_dims(input_mat, axis=0)
    input_ds_subset.write(input_mat)

    # Reproject a dataset
    transform_reproj, width_reproj, height_reproj = \
        calculate_default_transform(input_ds_subset.crs, output_crs,
                                    input_ds_subset.width, input_ds_subset.height, *input_ds_subset.bounds)
    kwargs_reproj = input_ds_subset.meta.copy()
    kwargs_reproj.update({
            'crs': output_crs,
            'transform': transform_reproj,
            'width': width_reproj,
            'height': height_reproj
        })

    output_ds = MemoryFile().open(**kwargs_reproj)
    reproject(source=rasterio.band(input_ds_subset, 1), destination=rasterio.band(output_ds, 1),
              src_transform=input_ds_subset.transform, src_crs=input_ds_subset.crs,
              dst_transform=transform_reproj, dst_crs=output_crs, resampling=Resampling.nearest)

    return output_ds

#### Pre-load variables:

In [4]:
# 0. Preload variables
path_smap = '../../Session_2A/2A.2_Precipitation-Analysis/Data'
path_shp = 'Data'
path_results = 'Outputs'

yearname = np.linspace(2016, 2021, 5, dtype='int')
monthnum = np.linspace(1, 12, 12, dtype='int')
monthname = np.arange(1, 13)
monthname = [str(i).zfill(2) for i in monthname]

In [5]:
# Generate a sequence of string between start and end dates (Year + DOY)
start_date = '2016-01-01'
end_date = '2021-12-31'
year = 2021 - 2016 + 1

start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
delta_date = end_date - start_date

date_seq = []
date_seq_doy = []
for i in range(delta_date.days + 1):
    date_str = start_date + datetime.timedelta(days=i)
    date_seq.append(date_str.strftime('%Y%m%d'))
    date_seq_doy.append(str(date_str.timetuple().tm_year) + str(date_str.timetuple().tm_yday).zfill(3))

In [6]:
# Count how many days for a specific year
yearname = np.linspace(2016, 2021, 6, dtype='int')
monthnum = np.linspace(1, 12, 12, dtype='int')
monthname = np.arange(1, 13)
monthname = [str(i).zfill(2) for i in monthname]

daysofyear = []
for idt in range(len(yearname)):
    if idt == 0:
        f_date = datetime.date(yearname[idt], monthnum[3], 1)
        l_date = datetime.date(yearname[idt], monthnum[-1], 31)
        delta_1y = l_date - f_date
        daysofyear.append(delta_1y.days + 1)
    else:
        f_date = datetime.date(yearname[idt], monthnum[0], 1)
        l_date = datetime.date(yearname[idt], monthnum[-1], 31)
        delta_1y = l_date - f_date
        daysofyear.append(delta_1y.days + 1)

daysofyear = np.asarray(daysofyear)

In [7]:
# Find the indices of each month in the list of days
nlpyear = 1999 # non-leap year
lpyear = 2000 # leap year

daysofmonth_nlp = np.array([calendar.monthrange(nlpyear, x)[1] for x in range(1, len(monthnum)+1)])
ind_nlp = [np.arange(daysofmonth_nlp[0:x].sum(), daysofmonth_nlp[0:x+1].sum()) for x in range(0, len(monthnum))]

daysofmonth_lp = np.array([calendar.monthrange(lpyear, x)[1] for x in range(1, len(monthnum)+1)])
ind_lp = [np.arange(daysofmonth_lp[0:x].sum(), daysofmonth_lp[0:x+1].sum()) for x in range(0, len(monthnum))]
ind_iflpr = np.array([int(calendar.isleap(yearname[x])) for x in range(len(yearname))]) # Find out leap years

# Generate a sequence of the days of months for all years
daysofmonth_seq = np.array([np.tile(daysofmonth_nlp[x], len(yearname)) for x in range(0, len(monthnum))])
daysofmonth_seq[1, :] = daysofmonth_seq[1, :] + ind_iflpr # Add leap days to February

daysofmonth_seq_cumsum = np.cumsum(daysofmonth_seq, axis=0)

#### Load geo-location parameters:

In [8]:
#os.chdir(path_code)
h5_path = '../../Session_2A/2A.2_Precipitation-Analysis/Data/ds_parameters.hdf5'
f = h5py.File(h5_path, "r")
varname_list = ['lat_world_ease_1km', 'lon_world_ease_1km']

for x in range(len(varname_list)):
    var_obj = f[varname_list[x]][()]
    exec(varname_list[x] + '= var_obj')
    del(var_obj)
f.close()

#### Working with SMAP Soil Moisture Data:

In [9]:
# 1. Read 1km SMAP SM data
daysofmonth_seq_cumsum_ind = np.copy(daysofmonth_seq_cumsum)
daysofmonth_seq_cumsum_ind[:, 0] = daysofmonth_seq_cumsum_ind[:, 0] - 90
daysofmonth_seq_cumsum_ind = [list(daysofmonth_seq_cumsum_ind[:, x]) for x in range(daysofmonth_seq_cumsum_ind.shape[1])]
del(daysofmonth_seq_cumsum_ind[0][0:3])
daysofmonth_seq_cumsum_ind = [[0] + daysofmonth_seq_cumsum_ind[x] for x in range(len(daysofmonth_seq_cumsum_ind))]

tif_files = sorted(glob.glob(path_smap + '/la_plata_1km_monthly/' + '*.tif'))
smap_sm_monthly_all = []
for ife in range(len(tif_files)):
    smap_sm_monthly = rasterio.open(tif_files[ife]).read()
    smap_sm_monthly_all.append(smap_sm_monthly)
    del(smap_sm_monthly)
    print(tif_files[ife])

../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_01.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_02.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_03.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_04.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_05.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_06.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_07.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_08.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_09.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_1km_monthly\smap_sm_1km_ds_2016_10.tif
../../Session_2A/2A.

In [10]:
# Subset and reproject the SMAP SM data at watershed
shp_file = path_shp + '/Parana_Sub-basin.shp'
shp_ds = ogr.GetDriverByName("ESRI Shapefile").Open(shp_file, 0)
shp_extent = list(shp_ds.GetLayer().GetExtent())
shapefile_read = fiona.open(shp_file, 'r')
crop_shape = [feature["geometry"] for feature in shapefile_read]
output_crs = 'EPSG:4326'

In [11]:
# 1.1 Load and Clip SM data by the profiles of La Plata river basin
[lat_1km, row_1km_ind, lon_1km, col_1km_ind] = \
    coordtable_subset(lat_world_ease_1km, lon_world_ease_1km,
                      shp_extent[3], shp_extent[2], shp_extent[1], shp_extent[0])

In [12]:
# 1 km data
masked_ds_1km_all = []
for iyr in range(len(smap_sm_monthly_all)):

    masked_ds_1km_1year = []
    for imo in range(len(smap_sm_monthly_all[iyr])):
        sub_window_1km = Window(col_1km_ind[0], row_1km_ind[0], len(col_1km_ind), len(row_1km_ind))
        kwargs_1km_sub = {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': len(lon_world_ease_1km),
                          'height': len(lat_world_ease_1km), 'count': 1, 'crs': CRS.from_dict(init='epsg:6933'),
                          'transform': Affine(1000.89502334956, 0.0, -17367530.44516138, 0.0, -1000.89502334956,
                                              7314540.79258289)}
        smap_sm_1km_output = sub_n_reproj(smap_sm_monthly_all[iyr][imo], kwargs_1km_sub, sub_window_1km, output_crs)

        masked_ds_1km, mask_transform_ds_1km = mask(dataset=smap_sm_1km_output, shapes=crop_shape, crop=True)
        masked_ds_1km[np.where(masked_ds_1km == 0)] = np.nan
        masked_ds_1km = masked_ds_1km.squeeze()

        masked_ds_1km_1year.append(masked_ds_1km)
        del(masked_ds_1km, smap_sm_1km_output)

    masked_ds_1km_all.append(masked_ds_1km_1year)
    print(iyr)
    del(masked_ds_1km_1year)

masked_ds_1km_all = np.asarray(masked_ds_1km_all)
masked_ds_1km_all = np.nanmean(masked_ds_1km_all, axis=1)
masked_ds_1km_all_avg = np.nanmean(np.nanmean(masked_ds_1km_all, axis=1), axis=1)

smap_divide_ind = np.arange(0, len(yearname)*12, 12)[1:]
#smap_divide_ind = smap_divide_ind - 3
masked_ds_1km_all_divide = np.split(masked_ds_1km_all, smap_divide_ind, axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


#### Make sublot for annual soil moisture maps:

In [17]:
title_content = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
feature_shp= ShapelyFeature(Reader(path_shp).geometries(),
                                ccrs.PlateCarree(), edgecolor='black', facecolor='none')

for iyr in range(len(yearname)):
    fig = plt.figure(figsize=(9, 6), facecolor='w', edgecolor='k', dpi=150)
    title_content_1year = title_content
    plt.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.9, hspace=0.2, wspace=0.2)
    for ipt in range(len(masked_ds_1km_all_divide[iyr])):
        ax = fig.add_subplot(3, len(masked_ds_1km_all_divide[iyr])//3, ipt+1, projection=ccrs.PlateCarree())
        ax.add_feature(feature_shp)
        img = ax.imshow(masked_ds_1km_all_divide[iyr][ipt, :, :], origin='upper', vmin=0, vmax=0.5, cmap='Spectral',
                    extent=shp_extent)
        gl = ax.gridlines(draw_labels=True, linestyle='--', linewidth=0.5, alpha=0.5, color='black')
        gl.xlocator = mticker.MultipleLocator(base=5)
        gl.ylocator = mticker.MultipleLocator(base=5)
        gl.xlabel_style = {'size': 6}
        gl.ylabel_style = {'size': 6}
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        ax.text(-49, -33, title_content_1year[ipt], fontsize=10, horizontalalignment='left',
                verticalalignment='top', weight='bold')

    cbar_ax = fig.add_axes([0.25, 0.04, 0.5, 0.015])
    cbar = plt.colorbar(img, cax=cbar_ax, extend='both', orientation='horizontal', pad=0.1)
    cbar.ax.tick_params(labelsize=7)
    cbar_ax.locator_params(nbins=5)
    cbar.set_label('$\mathregular{(m^3/m^3)}$', fontsize=7, x=1.08, y=0.05, labelpad=-15)
    plt.suptitle(yearname[iyr], fontsize=16, weight='bold')
    plt.savefig(path_results + '/sm_parana_' + str(yearname[iyr]) + '.png')
    plt.close()

#### Examine sample plot:

![test](Outputs/sm_parana_2021.png)

#### Make Monthly time-series plot:

In [20]:
tif_files = sorted(glob.glob(path_smap + '/la_plata_10km_prec_monthly/' + '*.tif'))
gpm_monthly_all = []
for ife in range(len(tif_files)):
    gpm_monthly = rasterio.open(tif_files[ife]).read()
    gpm_monthly_all.append(gpm_monthly)
    del(gpm_monthly)
    print(tif_files[ife])
gpm_monthly_all = np.stack(gpm_monthly_all, axis=0).squeeze()

gpm_monthly_all_avg = np.nanmean(np.nanmean(gpm_monthly_all, axis=1), axis=1)


fig = plt.figure(figsize=(15, 4), dpi=200)
plt.subplots_adjust(left=0.12, right=0.88, bottom=0.1, top=0.9, hspace=0.3, wspace=0.25)
x = np.concatenate([[np.nan, np.nan, np.nan], masked_ds_1km_all_avg])
z = np.concatenate([[np.nan, np.nan, np.nan], gpm_monthly_all_avg[0:]])
ax = fig.add_subplot(1, 1, 1)
lns1 = ax.plot(x, c='r', marker='s', label='SMAP', markersize=3, linestyle='--', linewidth=1)
#plt.show()
plt.xlim(0, len(x))
ax.set_xticks(np.arange(0, (len(yearname)+1)*12, 12))
ax.set_xticklabels([])
labels = ['2016', '2017', '2018', '2019', '2020', '2021']
mticks = ax.get_xticks()
ax.set_xticks((mticks[:-1] + mticks[1:]) / 2, minor=True)
ax.tick_params(axis='x', which='minor', length=0)
ax.set_xticklabels(labels, minor=True)

plt.ylim(0, 0.5)
ax.set_yticks(np.arange(0, 0.6, 0.1))
ax.tick_params(axis='y', labelsize=10)
ax.grid(linestyle='--')

ax2 = ax.twinx()
ax2.set_ylim(0, 30)
ax2.invert_yaxis()
ax2.set_yticks(np.arange(0, 72, 12))
lns4 = ax2.bar(np.arange(len(x)), z, width=0.8, color='cornflowerblue', label='Precipitation', alpha=0.5)
ax2.tick_params(axis='y', labelsize=10)
handles = lns1 + [lns4]
labels = [l.get_label() for l in handles]

plt.legend(handles, labels, loc=(0, 0.92), mode="expand", borderaxespad=0, ncol=4, prop={"size": 10})
fig.text(0.5, 0.01, 'Year', ha='center', fontsize=14, fontweight='bold')
fig.text(0.02, 0.2, 'SMAP SM ($\mathregular{m^3/m^3)}$', rotation='vertical', fontsize=14, fontweight='bold')
fig.text(0.96, 0.2, 'GPM Precipitation (mm)', rotation='vertical', fontsize=14, fontweight='bold')
plt.suptitle('Parana_Sub-basin', fontsize=16, y=0.98, fontweight='bold')
plt.savefig(path_results + '/smap_gpm_tseries_parana' + '.png')
plt.close(fig)

../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_01.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_02.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_03.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_04.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_05.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_06.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_07.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_08.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_monthly\gpm_prec_10km_2016_09.tif
../../Session_2A/2A.2_Precipitation-Analysis/Data/la_plata_10km_prec_mont

#### Examine figure:

![rainfall](Outputs/smap_gpm_tseries_parana.png)